# Data Cleaning for Ratings Exploration

## Importing Libraries

In [122]:
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import seaborn as sns
import os
from glob import glob
import plotly.express as px
%matplotlib inline

## Pulling in Previously Cleaned Data

In [123]:
# Pulling in our cleaned data, adding a profit column, and looking at head and info
df = pd.read_csv('cleaned_genre_exploration.csv')
df['profit'] = df['worldwide_gross']-df['production_budget']
#display(df.head())
#df.info()

In [124]:
#Pulling all the csv files into one list
csv_files = glob("./zippedData/*.csv.gz")
#and forming a dictionary of cleaned names
type(csv_files)
d = {}
for file in csv_files:
    d[file] = pd.read_csv(file)
    
csv_files_dict = {}
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_") # cleaning the filenames
    filename_df = pd.read_csv(filename, index_col=0)
    csv_files_dict[filename_cleaned] = filename_df

## Starting Our SQL Database

In [125]:
#Initializing connection:
conn = sqlite3.connect("movies_db.sqlite")

In [126]:
#Filling the DB with all the tables. . . 
"""
def create_sql_table_from_df(df, name, conn):
    try:
        df.to_sql(name, conn)
        print(f"Created table {name}")
    
    except Exception as e:
        print(f"could not make table {name}")
        print(e)
        
for name, table in csv_files_dict.items():
    create_sql_table_from_df(table, name, conn)
""";

In [127]:
#Showing names of tables
conn.execute("select name from sqlite_master where type='table';").fetchall()

[('bom_movie_gross_gz',),
 ('imdb_name_basics_gz',),
 ('imdb_title_akas_gz',),
 ('imdb_title_basics_gz',),
 ('imdb_title_crew_gz',),
 ('imdb_title_principals_gz',),
 ('imdb_title_ratings_gz',),
 ('tmdb_movies_gz',),
 ('tn_movie_budgets_gz',),
 ('clean_genre',)]

In [128]:
#Initializing cursor:
cur = conn.cursor()

In [129]:
#Adding my clean df to the sql db
##clean_genre = pd.read_csv('cleaned_genre_exploration.csv')
##clean_genre.to_sql('clean_genre', conn)

## Building Dataframes 

In [130]:
#We have a pretty solid data set here, but unreliable studio data
#Hopefully still enough to see who the top performers are. . . 
cur.execute("""SELECT production_budget, release_date, primary_title,
               runtime_minutes, G1, original_language, vote_count, vote_average,
               c.domestic_gross, worldwide_gross, m.studio
            
            FROM clean_genre as c
            LEFT JOIN bom_movie_gross_gz as m
            ON primary_title = m.title
            WHERE vote_count > 100
            ORDER BY vote_count DESC;""")

df3 = pd.DataFrame(cur.fetchall())
df3.columns = [i[0] for i in cur.description]
display(df3.head(2))
df3.info()

,production_budget,release_date,primary_title,runtime_minutes,G1,original_language,vote_count,vote_average,domestic_gross,worldwide_gross,studio
0,160000000,2010-07-16,Inception,148.0,Action,en,22186,8.3,292576195,835524642,WB
1,58000000,2016-02-12,Deadpool,108.0,Action,en,20175,7.6,363070709,801025593,Fox


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752 entries, 0 to 1751
Data columns (total 11 columns):
production_budget    1752 non-null int64
release_date         1752 non-null object
primary_title        1752 non-null object
runtime_minutes      1752 non-null float64
G1                   1752 non-null object
original_language    1752 non-null object
vote_count           1752 non-null int64
vote_average         1752 non-null float64
domestic_gross       1752 non-null int64
worldwide_gross      1752 non-null int64
studio               1304 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 150.7+ KB


In [131]:
#Taking the top 1000 by vote count seems to have helped the studio issue
cur.execute("""SELECT production_budget, release_date, primary_title,
               runtime_minutes, G1, original_language, vote_count, vote_average,
               c.domestic_gross, worldwide_gross, m.studio
            
            FROM clean_genre as c
            LEFT JOIN bom_movie_gross_gz as m
            ON primary_title = m.title
            WHERE vote_count > 100
            ORDER BY vote_count DESC
            LIMIT 1000;""")

df4 = pd.DataFrame(cur.fetchall())
df4.columns = [i[0] for i in cur.description]
display(df4.head())
df4.info()

,production_budget,release_date,primary_title,runtime_minutes,G1,original_language,vote_count,vote_average,domestic_gross,worldwide_gross,studio
0,160000000,2010-07-16,Inception,148.0,Action,en,22186,8.3,292576195,835524642,WB
1,58000000,2016-02-12,Deadpool,108.0,Action,en,20175,7.6,363070709,801025593,Fox
2,225000000,2012-05-04,The Avengers,143.0,Action,en,19673,7.6,623279547,1517935897,None
3,60000000,2012-05-04,The Avengers,143.0,Action,en,19673,7.6,23385416,48585416,None
4,165000000,2014-11-05,Interstellar,169.0,Adventure,en,18597,8.2,188017894,666379375,Par.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
production_budget    1000 non-null int64
release_date         1000 non-null object
primary_title        1000 non-null object
runtime_minutes      1000 non-null float64
G1                   1000 non-null object
original_language    1000 non-null object
vote_count           1000 non-null int64
vote_average         1000 non-null float64
domestic_gross       1000 non-null int64
worldwide_gross      1000 non-null int64
studio               814 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 86.1+ KB


In [132]:
#df4.corr()['vote_average'].sort_values()

In [133]:
#DF5 is a df grouped by studio so we can see info such as average rating,
#or average worldwide gross revenue.  
#This can help us identify successful studios!
cur.execute("""SELECT m.studio,
               AVG(production_budget) as avg_budg,
               SUM(production_budget) as tot_budg,
               AVG(runtime_minutes) as avg_runtime, 
               SUM(vote_count) as total_votes, 
               AVG(vote_average) as avg_rating,
               AVG(c.domestic_gross) as avg_dom_gross,
               SUM(c.domestic_gross) as tot_dom_gross,
               AVG(worldwide_gross) as avg_world_gross
            
            FROM clean_genre as c
            LEFT JOIN bom_movie_gross_gz as m
            ON primary_title = m.title
            WHERE vote_count > 100
            GROUP BY m.studio
            ORDER BY avg_rating DESC;""")

df5 = pd.DataFrame(cur.fetchall())
df5.columns = [i[0] for i in cur.description]
df5.dropna(axis = 0, subset = ['studio'], inplace = True)
display(df5.head())
df5.info()

,studio,avg_budg,tot_budg,avg_runtime,total_votes,avg_rating,avg_dom_gross,tot_dom_gross,avg_world_gross
0,Orch.,2.500000e+06,2500000,101.0,915,7.800000,5.205471e+06,5205471,2.384553e+07
1,Cleopatra,8.000000e+06,8000000,103.0,500,7.500000,8.270300e+04,82703,1.738269e+07
2,Neon,1.100000e+07,11000000,120.0,2904,7.500000,3.001453e+07,30014534,5.379741e+07
3,Osci.,4.333333e+06,13000000,106.0,1695,7.433333,1.796292e+06,5388877,6.200627e+06
4,MBox,4.630000e+07,92600000,120.0,4426,7.300000,5.317143e+07,106342853,1.273362e+08


<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 0 to 71
Data columns (total 9 columns):
studio             71 non-null object
avg_budg           71 non-null float64
tot_budg           71 non-null int64
avg_runtime        71 non-null float64
total_votes        71 non-null int64
avg_rating         71 non-null float64
avg_dom_gross      71 non-null float64
tot_dom_gross      71 non-null int64
avg_world_gross    71 non-null float64
dtypes: float64(5), int64(3), object(1)
memory usage: 5.5+ KB


In [176]:
#Here, we'll be pulling in info on actors and directors
cur.execute("""SELECT p.nconst, n.primary_name, p.category,
            SUM(c.domestic_gross) as tot_dom_gross,
            SUM(c.worldwide_gross) as tot_wor_gross,
            AVG(c.domestic_gross) as avg_dom_gross,
            AVG(c.worldwide_gross) as avg_wor_gross,
            AVG(c.vote_average) as avg_rating,
            AVG(c.production_budget) as avg_budg,
            COUNT(tconst) as num_movies
            FROM imdb_title_principals_gz as p
            INNER JOIN imdb_name_basics_gz as n
            USING (nconst)
            INNER JOIN clean_genre as c
            USING (tconst)
            WHERE category = 'director'
            GROUP BY nconst
            ORDER BY tot_dom_gross DESC
            LIMIT 100;""")

df6 = pd.DataFrame(cur.fetchall())
df6.columns = [i[0] for i in cur.description]
display(df6.head(10))
df6.info()

,nconst,primary_name,category,tot_dom_gross,tot_wor_gross,avg_dom_gross,avg_wor_gross,avg_rating,avg_budg,num_movies
0,nm0751577,Anthony Russo,director,1346646789,3902605502,4.488823e+08,1.300869e+09,7.800000,2.400000e+08,3
1,nm0751648,Joe Russo,director,1346646789,3902605502,4.488823e+08,1.300869e+09,7.800000,2.400000e+08,3
2,nm1349376,Francis Lawrence,director,1149112056,2543191543,2.298224e+08,5.086383e+08,6.880000,1.044000e+08,5
3,nm0269463,Jon Favreau,director,1142562693,2614568760,2.856407e+08,6.536422e+08,7.150000,1.128250e+08,4
4,nm0923736,Joss Whedon,director,1128220169,2992084614,2.820550e+08,7.480212e+08,7.425000,1.559000e+08,4
5,nm0634240,Christopher Nolan,director,1118801468,3086180484,2.797004e+08,7.715451e+08,7.900000,1.875000e+08,4
6,nm0719208,Chris Renaud,director,987963700,2405431942,3.293212e+08,8.018106e+08,6.733333,7.333333e+07,3
7,nm0811583,Zack Snyder,director,942497822,2420920114,1.884996e+08,4.841840e+08,6.220000,1.900000e+08,5
8,nm1853544,Pierre Coffin,director,884203670,2553409158,2.947346e+08,8.511364e+08,6.800000,7.333333e+07,3
9,nm0174374,Bill Condon,director,883325603,1873785010,2.944419e+08,6.245950e+08,7.100000,6.866667e+07,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
nconst           100 non-null object
primary_name     100 non-null object
category         100 non-null object
tot_dom_gross    100 non-null int64
tot_wor_gross    100 non-null int64
avg_dom_gross    100 non-null float64
avg_wor_gross    100 non-null float64
avg_rating       100 non-null float64
avg_budg         100 non-null float64
num_movies       100 non-null int64
dtypes: float64(4), int64(3), object(3)
memory usage: 7.9+ KB


## Exporting Our Dataframe

In [177]:
#Exporting the cleaned data to the repo
df6.to_csv('100_directors.csv')